# Silver Sales Transform — **retail_sales_analytics_and_forecasting**

_Minimal changes from healthcare reference; focused on Superstore sales._

## 1) Configuration

In [0]:

CATALOG = None
BRONZE_DB = "retail_sales_analytics_and_forecasting.bronze"
SILVER_DB = "retail_sales_analytics_and_forecasting.silver"
BRONZE_TBL = f"{BRONZE_DB}.superstore"
SILVER_TBL = f"{SILVER_DB}.superstore_clean"
print("Bronze:", BRONZE_TBL)
print("Silver:", SILVER_TBL)


Bronze: retail_sales_analytics_and_forecasting.bronze.superstore
Silver: retail_sales_analytics_and_forecasting.silver.superstore_clean


## 2) Spark session

In [0]:

try:
    spark  # type: ignore
    print("Using existing Spark session.")
except NameError:
    from pyspark.sql import SparkSession
    try:
        from delta import configure_spark_with_delta_pip
        builder = (
            SparkSession.builder.appName("retail_silver_minimal")
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        )
        spark = configure_spark_with_delta_pip(builder).getOrCreate()
        print("Started Spark with Delta via delta-spark.")
    except Exception as e:
        print("delta-spark not available, starting plain Spark:", e)
        spark = SparkSession.builder.appName("retail_silver_minimal_plain").getOrCreate()

if CATALOG:
    spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SILVER_DB}")


Using existing Spark session.


DataFrame[]

## 3) Load Bronze

In [0]:

df_bronze = spark.table(BRONZE_TBL)
print("Bronze rows:", df_bronze.count())
df_bronze.printSchema()
display(df_bronze.limit(5)) if 'display' in globals() else df_bronze.limit(5).toPandas()


Bronze rows: 9994
root
 |-- row_id: integer (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- ship_date: date (nullable = true)
 |-- ship_mode: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- segment: string (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: integer (nullable = true)
 |-- region: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- sales: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- discount: string (nullable = true)
 |-- profit: double (nullable = true)



row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,postal_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0,41.9136
2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0,219.582
3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0,6.8714
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.031
5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.368,2,0.2,2.5164


## 4) Minimal Silver transform

In [0]:

from pyspark.sql import functions as F
from pyspark.sql.window import Window

def trim_text(c):
    return F.regexp_replace(F.regexp_replace(F.trim(F.col(c)), r"\s+", " "), r"\u00A0", " ")

def safe_double(c):
    s = F.col(c).cast("string")
    cleaned = F.regexp_replace(s, r"[^0-9.\-+]", "")
    return F.when(cleaned.rlike(r"^[+-]?\d*(\.\d+)?$") & (F.length(cleaned) > 0), cleaned.cast("double")).otherwise(F.lit(None).cast("double"))

def safe_int(c):
    s = F.col(c).cast("string")
    cleaned = F.regexp_replace(s, r"[^0-9\-+]", "")
    return F.when(cleaned.rlike(r"^[+-]?\d+$") & (F.length(cleaned) > 0), cleaned.cast("int")).otherwise(F.lit(None).cast("int"))

df = df_bronze
df = df.withColumn("order_date", F.to_date(F.col("order_date").cast("string")))
df = df.withColumn("ship_date",  F.to_date(F.col("ship_date").cast("string")))

for c in ["sales","discount","profit"]:
    if c in df.columns: df = df.withColumn(c, safe_double(c))
if "quantity" in df.columns: df = df.withColumn("quantity", safe_int("quantity"))
if "postal_code" in df.columns: df = df.withColumn("postal_code", F.regexp_replace(F.col("postal_code").cast("string"), r"\.0$", ""))

for c in ["customer_name","product_name","city","state","country","category","sub_category","segment","ship_mode","region"]:
    if c in df.columns: df = df.withColumn(c, trim_text(c))

if "row_id" in df.columns:
    w = Window.partitionBy("row_id").orderBy(F.col("order_date").desc_nulls_last(), F.col("ship_date").desc_nulls_last())
    df = df.withColumn("rn", F.row_number().over(w)).filter(F.col("rn")==1).drop("rn")
else:
    w = Window.partitionBy("order_id","product_id").orderBy(F.col("order_date").desc_nulls_last())
    df = df.withColumn("rn", F.row_number().over(w)).filter(F.col("rn")==1).drop("rn")

df = (df
    .withColumn("order_year",  F.year("order_date"))
    .withColumn("order_month", F.month("order_date"))
    .withColumn("order_ym",    F.date_format("order_date", "yyyy-MM"))
    .withColumn("ship_days",   F.when(F.col("order_date").isNotNull() & F.col("ship_date").isNotNull(),
                                      F.datediff(F.col("ship_date"), F.col("order_date"))))
)

print("Transformed rows:", df.count())
display(df.limit(10)) if 'display' in globals() else df.limit(10).toPandas()


Transformed rows: 9994


row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,postal_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit,order_year,order_month,order_ym,ship_days
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136,2016,11,2016-11,3
2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0.0,219.582,2016,11,2016-11,3
3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0.0,6.8714,2016,6,2016-06,4
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.031,2015,10,2015-10,7
5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.368,2,0.2,2.5164,2015,10,2015-10,7
6,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,FUR-FU-10001487,Furniture,Furnishings,"Eldon Expressions Wood and Plastic Desk Accessories, Cherry Wood",48.86,7,0.0,14.1694,2014,6,2014-06,5
7,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,OFF-AR-10002833,Office Supplies,Art,Newell 322,7.28,4,0.0,1.9656,2014,6,2014-06,5
8,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,TEC-PH-10002275,Technology,Phones,Mitel 5320 IP Phone VoIP phone,907.152,6,0.2,90.7152,2014,6,2014-06,5
9,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,OFF-BI-10003910,Office Supplies,Binders,DXL Angle-View Binders with Locking Rings by Samsill,18.504,3,0.2,5.7825,2014,6,2014-06,5
10,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,OFF-AP-10002892,Office Supplies,Appliances,Belkin F5C206VTEL 6 Outlet Surge,114.9,5,0.0,34.47,2014,6,2014-06,5


## 5) Write Silver table

In [0]:

spark.sql(f"""CREATE TABLE IF NOT EXISTS {SILVER_TBL} (
  row_id BIGINT,
  order_id STRING,
  order_date DATE,
  ship_date DATE,
  ship_mode STRING,
  customer_id STRING,
  customer_name STRING,
  segment STRING,
  country STRING,
  city STRING,
  state STRING,
  postal_code STRING,
  region STRING,
  product_id STRING,
  category STRING,
  sub_category STRING,
  product_name STRING,
  sales DOUBLE,
  quantity INT,
  discount DOUBLE,
  profit DOUBLE,
  order_year INT,
  order_month INT,
  order_ym STRING,
  ship_days INT
) USING DELTA
""")
df.createOrReplaceTempView("silver_upsert_min")
spark.sql(f"""MERGE INTO {SILVER_TBL} AS tgt
USING silver_upsert_min AS src
ON tgt.row_id = src.row_id
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *
""")
display(spark.table(SILVER_TBL).limit(10)) if 'display' in globals() else spark.table(SILVER_TBL).limit(10).toPandas()


row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,postal_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit,order_year,order_month,order_ym,ship_days
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.000,41.91,2016,11,2016-11,3
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.58,5,0.450,-383.03,2015,10,2015-10,7
6,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,FUR-FU-10001487,Furniture,Furnishings,"Eldon Expressions Wood and Plastic Desk Accessories, Cherry Wood",48.86,7,0.000,14.17,2014,6,2014-06,5
8,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,TEC-PH-10002275,Technology,Phones,Mitel 5320 IP Phone VoIP phone,907.15,6,0.200,90.72,2014,6,2014-06,5
13,CA-2017-114412,2017-04-15,2017-04-20,Standard Class,AA-10480,Andrew Allen,Consumer,United States,Concord,North Carolina,28027,South,OFF-PA-10002365,Office Supplies,Paper,Xerox 1967,15.55,3,0.200,5.44,2017,4,2017-04,5
15,US-2015-118983,2015-11-22,2015-11-26,Standard Class,HP-14815,Harold Pawlan,Home Office,United States,Fort Worth,Texas,76106,Central,OFF-AP-10002311,Office Supplies,Appliances,"Holmes Replacement Filter for HEPA Air Cleaner, Very Large Room, HEPA Filter",68.81,5,0.800,-123.86,2015,11,2015-11,4
17,CA-2014-105893,2014-11-11,2014-11-18,Standard Class,PK-19075,Pete Kriz,Consumer,United States,Madison,Wisconsin,53711,Central,OFF-ST-10004186,Office Supplies,Storage,"""Stur-D-Stor Shelving, Vertical 5-Shelf: 72""""H x 36""""W x 18 1/2""""D""",665.88,6,0.000,13.32,2014,11,2014-11,7
20,CA-2014-143336,2014-08-27,2014-09-01,Second Class,ZD-21925,Zuschuss Donatelli,Consumer,United States,San Francisco,California,94109,West,TEC-PH-10001949,Technology,Phones,Cisco SPA 501G IP Phone,213.48,3,0.200,16.01,2014,8,2014-08,5
21,CA-2014-143336,2014-08-27,2014-09-01,Second Class,ZD-21925,Zuschuss Donatelli,Consumer,United States,San Francisco,California,94109,West,OFF-BI-10002215,Office Supplies,Binders,"Wilson Jones Hanging View Binder, White, 1""""",22.72,4,0.200,7.38,2014,8,2014-08,5
22,CA-2016-137330,2016-12-09,2016-12-13,Standard Class,KB-16585,Ken Black,Corporate,United States,Fremont,Nebraska,68025,Central,OFF-AR-10000246,Office Supplies,Art,Newell 318,19.46,7,0.000,5.06,2016,12,2016-12,4


## 6) Quick checks

In [0]:

spark.sql(f"SELECT COUNT(*) AS rows FROM {SILVER_TBL}").show()
spark.sql(f"""SELECT order_ym, ROUND(SUM(sales),2) AS sales, ROUND(SUM(profit),2) AS profit
FROM {SILVER_TBL}
GROUP BY order_ym ORDER BY order_ym LIMIT 12
""").show()
spark.sql(f"DESCRIBE {SILVER_TBL}").show(truncate=False)


+----+
|rows|
+----+
|9994|
+----+

+--------+--------+-------+
|order_ym|   sales| profit|
+--------+--------+-------+
| 2014-01|14261.35|2417.08|
| 2014-02| 9243.84| 786.04|
| 2014-03|55526.23| 419.42|
| 2014-04|28139.56|3453.58|
| 2014-05|23634.66|2732.61|
| 2014-06|34509.01|5058.27|
| 2014-07|33610.85|-925.08|
| 2014-08|28403.51|5353.99|
| 2014-09|81596.79|8215.76|
| 2014-10|34127.91|3453.89|
| 2014-11|84221.26|9182.21|
| 2014-12|72004.86|8913.11|
+--------+--------+-------+

+-------------+-------------+-------+
|col_name     |data_type    |comment|
+-------------+-------------+-------+
|row_id       |bigint       |NULL   |
|order_id     |string       |NULL   |
|order_date   |date         |NULL   |
|ship_date    |date         |NULL   |
|ship_mode    |string       |NULL   |
|customer_id  |string       |NULL   |
|customer_name|string       |NULL   |
|segment      |string       |NULL   |
|country      |string       |NULL   |
|city         |string       |NULL   |
|state        |string